In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import datetime
import time

browser = webdriver.Edge()
year1 = [2017+i for i in range(6)]
year2 = [2018+i for i in range(6)]
max_pages = 8
pages = [i+1 for i in range(max_pages)]
#league_names = ['ligue-1','ligue-2','national','coupe-de-france','division-1-women','division-2-women']
league_names = ['ligue-1','bundesliga','serie-a','premier-league','laliga']
country_names = ['france', 'germany','italy','england','spain']


TIME = []
MATCH_NAME = []
TABLE_SCORE = []
Cs = []
Bs = []
DATE = []
LEAGUE_NAME = []
COUNTRY_NAME = []
date_string = 'none'
for lc_indx in range(len(league_names)):
    for y in range(len(year1)):
        for pg in pages:

            url = 'https://www.oddsportal.com/soccer/'+country_names[lc_indx]+'/'+league_names[lc_indx]+'-'+str(year1[y])+'-'+str(year2[y])+'/results/#/page/'+str(pg)+'/'
            browser.get(url)
            time.sleep(2)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")


            if 'Page not found' in str(html):
                break;

            for table in soup.findAll('table',attrs={'class':'table-main','id':'tournamentTable'}):
                for tr in table.findAll('tr',attrs={'class':['deactivate','center nob-border']}):

                    th = tr.find('th')
                    if th==None:

                        td1 = tr.find('td',attrs={'class':'table-time'})
                        td2 = tr.find('td',attrs={'class':'name table-participant'})
                        a2 = td2.find('a')
                        td3 = tr.find('td',attrs={'class':'table-score'})    
                        tmp = []
                        for td4 in tr.findAll('td',attrs={'class':'odds-nowrp'}):
                            a4 = td4.find('a')
                            if a4==None:
                                tmp.append('-')
                            else:
                                tmp.append(a4.text)
                        td5 = tr.find('td',attrs={'class':'center info-value'})
                        TIME.append(td1.text)
                        MATCH_NAME.append(a2.text)
                        TABLE_SCORE.append(td3.text)
                        Cs.append(tmp)
                        Bs.append(td5.text)      
                        DATE.append(date_string)
                        LEAGUE_NAME.append(league_names[lc_indx])
                        COUNTRY_NAME.append(country_names[lc_indx])
                    else:
                        if 'Yesterday' in th.text:
                            date_string = th.text.split(' ')[1]+' '+th.text.split(' ')[2]+' '+str(datetime.datetime.now().year)
                        elif 'Today' in th.text:
                            continue
                        else:
                            date_string = th.text
browser.quit()

assert len(TIME)==len(MATCH_NAME)==len(TABLE_SCORE)==len(Cs)==len(Bs)==len(DATE)==len(LEAGUE_NAME)==len(COUNTRY_NAME)

In [ ]:
len(TIME)

9076

In [ ]:
import pandas as pd

df=pd.DataFrame()
df['COUNTRY'] = COUNTRY_NAME
df['LEAGUE_NAME'] = LEAGUE_NAME
df['DATE'] = DATE
df['DATE_DAY'] = [int(o.split(' ')[0]) for o in DATE]
df['DATE_MONTH'] = [o.split(' ')[1] for o in DATE]
df['DATE_YEAR'] = [int(o.split(' ')[2]) for o in DATE]
df['TIME'] = TIME
df['MATCH_NAME'] = MATCH_NAME
df['TEAM 1'] = [o.split('-')[0].strip() for o in MATCH_NAME]
df['TEAM 2'] = [o.split('-')[1].strip() for o in MATCH_NAME]
df['TABLE_SCORE'] = TABLE_SCORE
df['Cs'] = Cs
df['1'] = [o[0] for o in Cs]
df['X'] = [o[1] for o in Cs]
df['2'] = [o[2] for o in Cs]
df['Bs'] = Bs

df.to_csv('new_odds',index=False)

In [ ]:
df.head()

,COUNTRY,LEAGUE_NAME,DATE,DATE_DAY,DATE_MONTH,DATE_YEAR,TIME,MATCH_NAME,TEAM 1,TEAM 2,TABLE_SCORE,Cs,1,X,2,Bs
0,france,ligue-1,27 May 2018 - Relegation,27,May,2018,19:00,Toulouse - AC Ajaccio,Toulouse,AC Ajaccio,1:0,"[-139, +261, +405]",-139,+261,+405,10
1,france,ligue-1,23 May 2018 - Relegation,23,May,2018,18:45,AC Ajaccio - Toulouse,AC Ajaccio,Toulouse,0:3,"[+265, +212, +116]",+265,+212,+116,10
2,france,ligue-1,20 May 2018 - Relegation,20,May,2018,17:00,AC Ajaccio - Le Havre,AC Ajaccio,Le Havre,3:2 pen.,"[+125, +204, +244]",+125,+204,+244,10
3,france,ligue-1,19 May 2018,19,May,2018,19:00,Caen - Paris SG,Caen,Paris SG,0:0,"[+301, +254, -106]",+301,+254,-106,10
4,france,ligue-1,19 May 2018,19,May,2018,19:00,Dijon - Angers,Dijon,Angers,2:1,"[+129, +259, +200]",+129,+259,+200,10


In [ ]:
df = pd.read_csv("new_odds.csv")

In [ ]:
df.duplicated().sum()

0